# 0. Install and Import Dependencies

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [3]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# 2. Functions

In [4]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [8]:
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Recolor Feed
    #image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #image.flags.writeable = False
    
    # Recolor image back to BGR for rendering
    #image.flags.writeable = True
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #img = image.copy()
    #img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    #input_img = tf.cast(img, dtype=tf.int32)
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
keypoints_with_scores

array([[[ 7.18633831e-01,  5.22488415e-01,  6.39148295e-01],
        [ 6.47234499e-01,  5.69871843e-01,  5.08891940e-01],
        [ 6.34455860e-01,  4.88331050e-01,  5.46316564e-01],
        [ 6.72821879e-01,  6.27661169e-01,  4.61538136e-01],
        [ 6.42360389e-01,  4.44071978e-01,  4.53837633e-01],
        [ 9.07398403e-01,  6.92034662e-01,  7.40026161e-02],
        [ 9.51900363e-01,  3.49671215e-01,  1.25459120e-01],
        [ 9.73685384e-01,  6.98558271e-01,  2.21579685e-03],
        [ 9.72999454e-01,  3.42925131e-01,  1.11408001e-02],
        [ 9.81024921e-01,  6.13051355e-01,  4.64131609e-02],
        [ 9.18045521e-01,  3.94388527e-01,  8.47191364e-03],
        [ 1.00001192e+00,  6.05647683e-01,  1.41727983e-03],
        [ 9.92130399e-01,  4.43519920e-01,  8.13646242e-03],
        [ 9.62438703e-01,  5.51053464e-01,  2.66768533e-04],
        [ 8.22580099e-01,  4.41478461e-01,  4.98180697e-03],
        [ 3.74264538e-01,  5.21388113e-01,  3.44629656e-03],
        [ 4.63673353e-01

In [ ]:
practice

In [45]:
sample = np.squeeze(np.multiply(,))

In [48]:
np.delete(sample, 2, 2)

array([[[0.62932986, 0.55529183],
        [0.5684689 , 0.5958175 ],
        [0.57016677, 0.5255055 ],
        [0.61493576, 0.63896257],
        [0.6049543 , 0.4908645 ],
        [0.90824044, 0.7380211 ],
        [0.89123935, 0.40378928],
        [0.97709084, 0.7748437 ],
        [0.9758281 , 0.3888321 ],
        [0.9320585 , 0.6702027 ],
        [0.96477884, 0.43738127],
        [0.9980997 , 0.61886835],
        [0.97179115, 0.51751107],
        [0.9291886 , 0.6138124 ],
        [0.9192876 , 0.4225894 ],
        [0.38787267, 0.5940819 ],
        [0.48470208, 0.49625406]],

       [[0.07218732, 0.13396674],
        [0.07431567, 0.13356254],
        [0.0715057 , 0.13608642],
        [0.0786258 , 0.13089404],
        [0.07344069, 0.13661276],
        [0.07987447, 0.13006099],
        [0.07213606, 0.13379513],
        [0.06636928, 0.13461028],
        [0.06302733, 0.13591036],
        [0.06892523, 0.13597998],
        [0.06746319, 0.13669424],
        [0.07308623, 0.12712923],
        [0.0